In [2]:
import networkx as nx
import os
import random

# Asegura que las carpetas necesarias existen
def crear_estructura_directorios():
    base = "ARCHIVOS_REDES"
    subdirs = ["ER", "WS", "ER_parametros", "WS_parametros"]
    for sub in subdirs:
        os.makedirs(os.path.join(base, sub), exist_ok=True)

crear_estructura_directorios()

def contar_archivos_en_directorio(directorio):
    return len([f for f in os.listdir(directorio) if f.endswith(".txt")])


def guardar_lista_adyacencia(grafo, nombre_base, modelo, parametros):
    """
    Guarda el grafo en formato lista de adyacencia y los parámetros usados.

    Args:
        grafo (nx.Graph): Grafo a guardar.
        nombre_base (str): Nombre base para el archivo.
        modelo (str): 'ER' o 'WS'.
        parametros (dict): Parámetros a guardar en el archivo de parámetros.
    """
    carpeta_grafos = os.path.join("ARCHIVOS_REDES", modelo)
    carpeta_params = os.path.join("ARCHIVOS_REDES", f"{modelo}_parametros")

    i = contar_archivos_en_directorio(carpeta_grafos)
    nombre_grafo = f"{nombre_base}_{i}.txt"
    nombre_param = f"{nombre_base}_{i}.txt"

    # Guardar lista de adyacencia
    with open(os.path.join(carpeta_grafos, nombre_grafo), 'w') as f:
        for nodo in sorted(grafo.nodes()):
            vecinos = sorted(grafo.neighbors(nodo))
            for vecino in vecinos:
                f.write(f"{nodo} {vecino}\n")

    # Guardar parámetros
    with open(os.path.join(carpeta_params, nombre_param), 'w') as f:
        for clave, valor in parametros.items():
            f.write(f"{clave}: {valor}\n")


def generar_red_ER_conexa(N, k_prom, aceptancia_N, aceptancia_k, nombre_base="ER"):
    p = k_prom / (N - 1)

    while True:
        G = nx.erdos_renyi_graph(N, p)
        if not nx.is_connected(G):
            giant = max(nx.connected_components(G), key=len)
            G = G.subgraph(giant).copy()

        N_real = G.number_of_nodes()
        k_real = sum(dict(G.degree()).values()) / N_real

        if (abs(N_real - N) / N * 100 <= aceptancia_N and
                abs(k_real - k_prom) / k_prom * 100 <= aceptancia_k):
            parametros = {
                "N_ideal": N,
                "N_real": N_real,
                "k_prom_ideal": k_prom,
                "k_prom_real": round(k_real, 3)
            }
            guardar_lista_adyacencia(G, nombre_base, "ER", parametros)
            return G, N_real, k_real


def generar_red_WS(N, k, p, nombre_base="WS"):
    G = nx.watts_strogatz_graph(N, k, 0)
    feos = []
    guapos = []

    for edge in G.edges():
        if random.random() < p:
            feos.append(edge)
        else:
            guapos.append(edge)

    while feos:
        random.shuffle(feos)
        enlace0 = feos[0]
        nodo0_a, nodo0_b = enlace0
        encontrado = False

        for i in range(1, len(feos)):
            enlace1 = feos[i]
            nodo1_a, nodo1_b = enlace1
            if nodo0_a not in enlace1 and nodo0_b not in enlace1:
                nodos1 = [nodo1_a, nodo1_b]
                random.shuffle(nodos1)
                A = (nodo0_a, nodos1[0])
                B = (nodo0_b, nodos1[1])
                if A not in guapos and (A[1], A[0]) not in guapos and B not in guapos and (B[1], B[0]) not in guapos:
                    guapos.append(A)
                    guapos.append(B)
                    del feos[i]
                    del feos[0]
                    encontrado = True
                    break

        if not encontrado:
            guapos.append(feos[0])
            del feos[0]

    G_guapo = nx.Graph()
    G_guapo.add_nodes_from(range(N))
    G_guapo.add_edges_from(guapos)

    parametros = {
        "N": N,
        "k": k,
        "p": p
    }

    guardar_lista_adyacencia(G_guapo, nombre_base, "WS", parametros)
    return G_guapo


In [4]:
from datetime import datetime

def escribir_en_historial(nombre_archivo, parametros, archivo_inicio, archivo_fin):
    """
    Escribe una entrada en el historial correspondiente.

    Args:
        nombre_archivo (str): Nombre del archivo de historial (ER o WS).
        parametros (dict): Parámetros usados en la función.
        archivo_inicio (str): Nombre del primer archivo creado.
        archivo_fin (str): Nombre del último archivo creado.
    """
    ruta = os.path.join("ARCHIVOS_REDES", nombre_archivo)
    with open(ruta, 'a') as f:
        f.write("---------------------------------------------------------------------------------\n")
        f.write(f"Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write("Parámetros:\n")
        for clave, valor in parametros.items():
            f.write(f"{clave}: {valor}\n")
        f.write(f"Archivo inicial generado: {archivo_inicio}\n")
        f.write(f"Archivo final generado: {archivo_fin}\n")
        f.write("---------------------------------------------------------------------------------\n\n")


def generar_varias_redes_ER(N_redes, N_nodos, k_prom, aceptancia_N, aceptancia_k, nombre_base="ER"):
    carpeta_grafos = os.path.join("ARCHIVOS_REDES", "ER")
    i_inicial = contar_archivos_en_directorio(carpeta_grafos)

    for _ in range(N_redes):
        generar_red_ER_conexa(N_nodos, k_prom, aceptancia_N, aceptancia_k, nombre_base)

    i_final = i_inicial + N_redes - 1
    archivo_inicio = f"{nombre_base}_{i_inicial}.txt"
    archivo_fin = f"{nombre_base}_{i_final}.txt"

    parametros_historial = {
        "N_redes": N_redes,
        "N_nodos": N_nodos,
        "k_prom": k_prom,
        "aceptancia_N (%)": aceptancia_N,
        "aceptancia_k (%)": aceptancia_k
    }

    escribir_en_historial("Historial_ER.txt", parametros_historial, archivo_inicio, archivo_fin)


def generar_varias_redes_WS(N_redes, N_nodos, k, p, nombre_base="WS"):
    carpeta_grafos = os.path.join("ARCHIVOS_REDES", "WS")
    i_inicial = contar_archivos_en_directorio(carpeta_grafos)

    for _ in range(N_redes):
        generar_red_WS(N_nodos, k, p, nombre_base)

    i_final = i_inicial + N_redes - 1
    archivo_inicio = f"{nombre_base}_{i_inicial}.txt"
    archivo_fin = f"{nombre_base}_{i_final}.txt"

    parametros_historial = {
        "N_redes": N_redes,
        "N_nodos": N_nodos,
        "k": k,
        "p": p
    }

    escribir_en_historial("Historial_WS.txt", parametros_historial, archivo_inicio, archivo_fin)

In [5]:
generar_varias_redes_WS(100,1000,4,0.5)

In [18]:
!git add .


In [19]:
!git commit -m "Mejora de funciones y eliminación de archivos basura"


[main 28f8b35] Mejora de funciones y eliminaciÃ³n de archivos basura
 1 file changed, 35 insertions(+), 16 deletions(-)


In [9]:
!git config --global user.name "DonIgnacioHerraiz"
!git config --global user.email "874462@unizar.es"


In [10]:
!git config --list


diff.astextplain.textconv=astextplain
filter.lfs.clean=git-lfs clean -- %f
filter.lfs.smudge=git-lfs smudge -- %f
filter.lfs.process=git-lfs filter-process
filter.lfs.required=true
http.sslbackend=openssl
http.sslcainfo=C:/Program Files/Git/mingw64/etc/ssl/certs/ca-bundle.crt
core.autocrlf=true
core.fscache=true
core.symlinks=false
pull.rebase=false
credential.helper=manager
credential.https://dev.azure.com.usehttppath=true
init.defaultbranch=main
core.editor="C:\Users\USUARIO\Desktop\Segundo semeste 2324\Computacional\Microsoft VS Code\bin\code" --wait
user.name=DonIgnacioHerraiz
user.email=874462@unizar.es
core.repositoryformatversion=0
core.filemode=false
core.bare=false
core.logallrefupdates=true
core.symlinks=false
core.ignorecase=true
remote.origin.url=https://github.com/872943/CaOtIcOs.git
remote.origin.fetch=+refs/heads/*:refs/remotes/origin/*
branch.main.remote=origin
branch.main.merge=refs/heads/main


In [11]:
!git commit -m "Mejoras"


On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   ../.ipynb_checkpoints/CODIGO_REDES-checkpoint.ipynb
	modified:   ../CODIGO_REDES.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [20]:
!git push


To https://github.com/872943/CaOtIcOs.git
 ! [rejected]        main -> main (fetch first)
error: failed to push some refs to 'https://github.com/872943/CaOtIcOs.git'
hint: Updates were rejected because the remote contains work that you do not
hint: have locally. This is usually caused by another repository pushing to
hint: the same ref. If you want to integrate the remote changes, use
hint: 'git pull' before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


In [21]:
!git pull --rebase



error: cannot pull with rebase: You have unstaged changes.
error: Please commit or stash them.
